In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error

print("Bibliotecas OK")


Bibliotecas OK


In [11]:
import numpy as np
import pandas as pd

# reprodutibilidade
np.random.seed(42)

n = 100
temperatura_c = np.random.uniform(15, 40, n).round(1)  # 15°C a 40°C
# relação linear simples: quanto mais quente, mais vende (com ruído)
vendas = (50 + 20*temperatura_c + np.random.normal(0, 30, n)).round()
vendas = np.clip(vendas, 0, None).astype(int)

df = pd.DataFrame({"temperatura_c": temperatura_c, "vendas": vendas})

print(df.shape)
pd.set_option('display.max_rows', None)
df



(100, 2)


,temperatura_c,vendas
0,24.4,541
1,38.8,817
2,33.3,719
3,30.0,590
4,18.9,421
5,18.9,439
6,16.5,424
7,36.7,768
8,30.0,626
9,32.7,689


In [5]:
# Dividir em treino e teste
X = df[["temperatura_c"]]
y = df["vendas"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinar o modelo
modelo = LinearRegression()
modelo.fit(X_train, y_train)

# Avaliar
y_pred = modelo.predict(X_test)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"Coeficiente angular (inclinação): {modelo.coef_[0]:.2f}")
print(f"Intercepto: {modelo.intercept_:.2f}")
print(f"R²: {r2:.3f}")
print(f"Erro médio absoluto (MAE): {mae:.2f}")


Coeficiente angular (inclinação): 19.51
Intercepto: 61.64
R²: 0.975
Erro médio absoluto (MAE): 17.80


In [12]:
# Teste de previsão com todas as temperaturas do dataset
previsoes = modelo.predict(df[["temperatura_c"]])

resultado = pd.DataFrame({
    "temperatura_c": df["temperatura_c"],
    "vendas_previstas": previsoes.round().astype(int)
})

pd.set_option('display.max_rows', None)  # mostra todas as 100 linhas
print(resultado)



    temperatura_c  vendas_previstas
0            24.4               538
1            38.8               819
2            33.3               711
3            30.0               647
4            18.9               430
5            18.9               430
6            16.5               384
7            36.7               778
8            30.0               647
9            32.7               700
10           15.5               364
11           39.2               827
12           35.8               760
13           20.3               458
14           19.5               442
15           19.6               444
16           22.6               503
17           28.1               610
18           25.8               565
19           22.3               497
20           30.3               653
21           18.5               423
22           22.3               497
23           24.2               534
24           26.4               577
25           34.6               737
26           20.0           

In [7]:
import joblib
import os

# Criar pasta de saída
os.makedirs("model", exist_ok=True)

# Salvar o modelo treinado
joblib.dump(modelo, "model/modelo_sorvete.pkl")

print("Modelo salvo com sucesso!")


Modelo salvo com sucesso!


In [8]:
from azureml.core import Workspace, Model

# Conectar ao workspace atual
ws = Workspace.from_config()

# Registrar o modelo
model = Model.register(
    workspace=ws,
    model_path="model/modelo_sorvete.pkl",
    model_name="modelo_sorvete_v1",
    description="Modelo de previsão de vendas de sorvete baseado na temperatura"
)

print(f"Modelo registrado com sucesso: {model.name}, versão {model.version}")


Registering model modelo_sorvete_v1
Modelo registrado com sucesso: modelo_sorvete_v1, versão 1
